In [ ]:
# basic imports
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path
import warnings

# morphopy imports
from morphopy.neurontree import NeuronTree as nt
from morphopy.computation.feature_presentation import compute_morphometric_statistics


Goal: is to extract features from the neuron tree and store them in a pandas dataframe.

Input: pickle file containing a dict of neuron_name: swc data

Steps:
- load the pickle file
- for each neuron:
    - create a NeuronTree object
    - compute morphometric features
    - add neuron name and pkl filename to the dataframe
- (optional, default yes) export the dataframe to a csv file
    - otherwise, return the dataframe


Desired output:
  - DataFrame with calculated morphometric features
    - should add columns to include neuron name and pkl file name


In [ ]:
swc_path = Path(
    "/Users/kyle/Library/CloudStorage/Box-Box/FoxLab/data-microscopy/neuromorpho-benchmark/swc_data"
)

dopamine_swc_pkl = next(swc_path.glob("*dopamine*.pkl"))
swc_dict = pd.read_pickle(dopamine_swc_pkl)

In [ ]:
tmp_dict = {n:swc_dict[n] for n in list(swc_dict.keys())[0:10]}

In [ ]:
def compute_morphopy_features(neuron_swc: pd.DataFrame) -> pd.DataFrame:
    """Compute morphometric features using MorphoPy.

    Returns:
        morphology_features: DataFrame of morphology features.
    """
    if "id" in neuron_swc.columns:
        neuron_swc.rename(columns={"id": "n"}, inplace=True)
    
    neuron_tree = nt.NeuronTree(neuron_swc)
    morphopy_features = compute_morphometric_statistics(neuron_tree)
    
    return morphopy_features

In [ ]:
def batch_process_morphopy_features(swc_dict: dict) -> pd.DataFrame:
    """Batch process morphometric features using MorphoPy.

    Args:
        swc_dict (dict): dictionary of swc data.


    Returns:
        morphology_features: DataFrame of morphology features.
    """
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    morpho_data_list = []
    for neuron, swc_data in swc_dict.items():
        try:
            morpho_data = compute_morphopy_features(swc_data)
            morpho_data["neuron_name"] = neuron
            morpho_data_list.append(morpho_data)
        except ValueError:
            print(f"Error processing {neuron}! Skipping...")

    return pd.concat(morpho_data_list, axis=0)


def export_features_from_pkl(pkl_path: Path | str, export_file: str) -> None:
    """Export morphometric features from pkl file to csv in the same directory.

    Args:
        pkl_path (Path): path to pkl file of swc data.
        export_file (str): name of exported file.
    """
    pkl_path = Path(pkl_path)
    print(f"Starting features extraction from {Path(pkl_path.name)}...")
    swc_dict = pd.read_pickle(pkl_path)
    morpho_features = batch_process_morphopy_features(swc_dict)
    morpho_features["filename"] = pkl_path.name
    export_file = export_file if Path(export_file).suffix == ".csv" else f"{export_file}.csv"
    morpho_features.to_csv(f"{pkl_path.parent}/{export_file}", index=False)
    print(f"Finished! Exported features to {pkl_path.parent}/{export_file}.")


In [ ]:
cell_types = ["neoctx-hpc_interneuron", "neoctx-pyramidal"]

for cell in cell_types:
    pkl = next(swc_path.glob(f"*{cell}*.pkl"))
    export_features_from_pkl(pkl, f"{cell}_morphopy_features.csv")

In [ ]:
cell_types = ["neoctx-pyramidal"]

for cell in cell_types:
    pkl = next(swc_path.glob(f"*{cell}*.pkl"))
    export_features_from_pkl(pkl, f"{cell}_morphopy_features.csv")